# Similarity Compared
----

In this notebook we will execute all the link prediction algorithms available in cuGraph and compare the results. These algorithms by default look at each possible pair of vertices in the graph and compare them based on the number of neigbors they share in common normalized by differing calculations of their individual neigborhoods :

| Author Credit |    Date    |  Update          | cuGraph Version |  Test Hardware        |
| --------------|------------|------------------|-----------------|-----------------------|
| Don Acosta    | 09/21/2023 | created          | 23.10 nightly   | AMPERE A6000 CUDA 11.7|


**Note: On large graphs these algorithms can take prohibitive time or memory. The notebook will show how to run on defined pairs instead.**

Load the required dependencies.

In [ ]:
import cugraph
from cugraph.datasets import dining_prefs
# only needed to display results in a table 
from IPython.display import display_html 

Function that calls all the cuGraph similarity/link prediction algorithms 

In [ ]:
def compute_similarity(G,pairs=None):
    _jdf = cugraph.jaccard(G,pairs)
    _odf = cugraph.overlap(G,pairs)
    _sdf = cugraph.sorensen_coefficient(G,pairs)
    return _jdf, _odf, _sdf

Function to put all the results in a convenient table

In [ ]:
# Print function
def print_similarity(jdf,odf,sdf,num_records=5):

    js_top = jdf.sort_values(by='jaccard_coeff', ascending=False).head(num_records).to_pandas()
    os_top = odf.sort_values(by='overlap_coeff', ascending=False).head(num_records).to_pandas()
    ss_top = sdf.sort_values(by='sorensen_coeff', ascending=False).head(num_records).to_pandas()
    
    df1_styler = js_top.style.set_table_attributes("style='display:inline'").set_caption('Jaccard').hide(axis='index')
    df2_styler = os_top.style.set_table_attributes("style='display:inline'").set_caption('Overlap').hide(axis='index')
    df3_styler = ss_top.style.set_table_attributes("style='display:inline'").set_caption('Sørensen').hide(axis='index')

    display_html(df1_styler._repr_html_()+df2_styler._repr_html_()+df3_styler._repr_html_(), raw=True)

In [ ]:
import cugraph
from cugraph.datasets import dining_prefs
gdf = dining_prefs.get_edgelist()
G = cugraph.Graph()
G = cugraph.from_cudf_edgelist(gdf, source='src', destination='dst', edge_attr = 'wgt')
jdf = cugraph.jaccard(G)
odf = cugraph.overlap(G)
sdf = cugraph.sorensen_coefficient(G)
print(jdf.head())
print(odf.head())
print(sdf.head())

Create the graph from the Dining preferences data set.

In [ ]:
G = dining_prefs.get_graph()

Run the three similarity Algorithms and print out the five links with the highest scores.

In [ ]:
jdf, odf, sdf = compute_similarity(G)
print_similarity(jdf,odf,sdf)

Now find the the complete set of two-hop neigbors and compare them instead of just using the existion one-hop edges. In a larger graph, this will run considerably faster since the default 

In [ ]:
# this cugraph algorithm pulls a set containing every pair of vertices
# that are within 2-hops of each other
two_hops_pairs = G.get_two_hop_neighbors()

jdf_hops, odf_hops, sdf_hops = compute_similarity(G,pairs=two_hops_pairs)
print_similarity(jdf_hops,odf_hops,sdf_hops)

---
### It's that easy with cuGraph

Copyright (c) 2023, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License");  you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
___